In [6]:
import cv2 
import numpy as np
import os
from pathlib import Path
import time
from imutils import paths
import keras
from PIL import Image
from tensorflow import random
from keras.preprocessing import image 
from keras.models import load_model
from keras.applications.vgg16 import decode_predictions


?



#global_variables
total = 3
home = os.getcwd()
threshold =350
DIR = '../5.1 masteropencv4_v1.5/images/'#directory to store registered mugshots
DIR2='/5.1 masteropencv4_v1.5/images/'
protoPath = "../Downloads/face_detection_model/deploy.prototxt"
modelPath = '../Downloads/face_detection_model/res10_300x300_ssd_iter_140000.caffemodel'


#main function:abstraction to the other functions
def image_registry(uploaded_image):

    process_1=mugshot_checker(uploaded_image)
    
    if process_1 == 1:
        registry_status(uploaded_image)
      
    else:
        msg = print('image not  a mugshot ,please upload a mugshot,check sample images')
        return msg
        
    
        
        
        
        

#mugshot checker checks if image uploaded is a mugshot or not and returns mugshot status
           
def mugshot_checker(new_image):
    print('mugshot check started')
    image = cv2.imread(new_image)
    MODEL = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123, 0))
    MODEL.setInput(blob)
    detections = MODEL.forward()  
    COUNT = 0
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.165:
            COUNT = COUNT + 1
    print('mugshot check completed')
    return COUNT



#feature matching fuction uses the orb algo to compare images and returns the threshold value
def feature_matcher(new_image,registered_img):
    k1 =new_image
    k2=registered_img
    new_image=cv2.imread(new_image)
    registered_img=cv2.imread(registered_img)
    print('feature_matching started')
    #preprocess the loaded images for Registry status function
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    Registered_img = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)
    # initaialize the orb feature matcher    
    orb = cv2.ORB_create(1000, 1.2)
    #detect and compute keypoints 
    kp1, des1 = orb.detectAndCompute(image1, None)
    kp2, des2 = orb.detectAndCompute(Registered_img, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    #compute the keypoint difference
    matches = bf.match(des1,des2)   
    matches = sorted(matches, key=lambda val: val.distance)
    print(len(matches))
    if len(matches) < 350:
#         test=registry_validator(k1,k2)
          return 0
        
    
        
     
    else:
        print('image_registered')
        
        
    


def registry_validator(new_image,new_image1):
    print('registry validation started')
    img =new_image1
    classifier = keras.applications.vgg16.VGG16() 
    target_size = (224, 224)
    new_image = image.load_img(new_image, target_size = (224, 224)) 
    new_image1 = image.load_img(new_image1, target_size = (224, 224))
    new_image = image.img_to_array(new_image)
    new_image = np.expand_dims(new_image, axis =0)
    new_image1 = image.img_to_array(new_image1)
    new_image1 = np.expand_dims(new_image1, axis =0)
    result1 = classifier.predict(new_image)
    result2 = classifier.predict(new_image1)
    if result1[0][0] ==  result2[0][0]:
#         output =print('Image registered')
          output=1
    else:
#         print('saving_image')
#         p = home + DIR2 + str(total).zfill(5) + '.jpg'
#         Image.open(img).save(p) 
#         print('image_saved')
#         
# #                 total += 1
# #         output = print('image_not registered')
        output =0
    return output
    print('validation_ended')
    






#registry status function checks if mugshot from mugshot checker func is registered or not and saves it
def registry_status(new_image):
    print('registry status check started')
    #read in the image
    imgsaver=cv2.imread(new_image)#assign the read image to a variable
    img = new_image #store the image name/dir to img variable
      #initialize a counter to name the registerd images
#     filename, file_extension = os.path.splitext(new_image)#split the name of the image into name +file extension
    start_time = time.perf_counter()#track feature matching time
# #     print(start_time)
#     pathlist = Path('..Downloads/mugshot_images/test_set/not_mugshot/').glob('**/*.jpg')#initialize the directory for the registered images 
#     #loop through the registered images 
#     for path in pathlist:
#         if(file_extension in ['.png', '.jpg', '.pgm', '.jpeg']):#check the file extensions format
#             path = str(path) #convert the path name to a string
    for filename in os.listdir('../Downloads/5.1 masteropencv4_v1.5/images/'):
        if filename.endswith('.jpg'):
            path =(os.path.join('../Downloads/5.1 masteropencv4_v1.5/images/',filename))
            path =str(path)
            img1 = path
            matchtest=registry_validator(img1,img)#pass in the path and imgsaver variables to the feature match fxn
            if matchtest == 1 :
                print('profile image already registered ')
                break
#         print('profile image saved ,profile_not_registered,proceed to register')
    else:
        print('no_match')
        total =10
        p = home + DIR2 + str(total).zfill(5) + '.jpg'
        Image.open(img).save(p) 
        total += 1
        print('profile image saved ,profile_not_registered,proceed to register')
    
                
                    

    
    



#optimize the process time with an image resizing in the mugshot checker function            
#  make sure you tune the threshold  value to be able to determine if image is registered
# add other feature matching models to validate if image has been regidtered or not
#adding more feature matching  models will increase the computer time for cpus and less for Gpus     
# a ='../5.1 masteropencv4_v1.5/images/kim.jpg'      
# a='C:/Users/kapp/Desktop/models and project/px-face-recognition/dataset/unknown/alan_grant.jpg'
# b= 'C:/Users/kapp/Desktop/models and project/px-face-recognition/dataset/Fergusson/00002.jpg'
# c= 'C:/Users/kapp/Downloads/mugshot_images/october07201113.jpg'
# c= 'C:/Users/kapp/Downloads/mugshot_images/22.jpg'
c= 'C:/Users/kapp/Downloads/5.1 masteropencv4_v1.5/images/gt.jpg'


y =image_registry(c)
# y =registry_status(c)

y


mugshot check started
mugshot check completed
registry status check started
registry validation started
registry validation started
profile image already registered 


In [ ]:
u='C:/Users/kapp/Desktop/models and project/px-face-recognition/dataset/ernest/00005.png'
b='../5.1 masteropencv4_v1.5/images/obama.jpg' 
x,y=os.path.splitext(u)
x
# h=x.split("\\")
# h[0]